In [25]:
# from langchain_community.document_loaders import WebBaseLoader, AsyncHtmlLoader

# loader = WebBaseLoader("https://www.espn.com/")
# loader.requests_kwargs = {"verify": False}

# docs = loader.load()

In [26]:
# from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


# from bs4 import BeautifulSoup as Soup

# url = "https://huggingface.co/"
# loader = RecursiveUrlLoader(
#     url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
# )
# docs = loader.load()



In [2]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import nest_asyncio
page = None
async def main(url):
    global page
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    
    # Get the page content after JavaScript execution
    page_content = await page.content()

    # Close the browser
    await browser.close()

    # Now you can use BeautifulSoup to parse the page content
    soup = BeautifulSoup(page_content, 'html.parser')

    # Perform scraping using BeautifulSoup on the dynamically rendered content
    # ...return soup
    # print(soup.prettify())
    return soup

nest_asyncio.apply()
url = "https://www.mdcsoftware.com.vn/"
soup = asyncio.get_event_loop().run_until_complete(main(url))



In [15]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text(soup.text[:12] + soup.text[58:])

chunk_size = 200
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(html_header_splits)

In [16]:
splits

[Document(page_content='MDC SoftwareHomeProductServicesRecruitmentContactEnglishOur specialized teamsWe combine strategy, technology, data science and design to solve the most complex business challenges.Scroll to see'),
 Document(page_content='challenges.Scroll to see moreWHO ARE WE?Our specialized teams lead the product design and development process for high growth companies.At MDC, we always appreciate the smallest ideas, you will be'),
 Document(page_content='smallest ideas, you will be completely assured with a clear, fast workflow. While our project consultancy is completely free of charge, we offer extensive QA process of highest quality to our clients'),
 Document(page_content='quality to our clients with reasonable price. Our experienced team of engineers are always enthusiastic in taking up new challenges and offering our clients with the best service. In addition, each'),
 Document(page_content='service. In addition, each project is carefully planned to stay consistent with

In [8]:
import openai
from dotenv import load_dotenv, find_dotenv
import os
import sys

sys.path.append("../..")


_ = load_dotenv(find_dotenv())  # read local .env file

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

openai.api_key = os.environ["OPENAI_API_KEY"]
model = os.environ["OPENAI_MODEL"]

In [9]:
from agentic_chunking_copy import AgenticChunker

In [11]:
ac = AgenticChunker(model_name=model)
ac.add_propositions(chunks)


Adding: 'MDC Software'
No chunks, creating a new one
Created new chunk (e143b): Software Development

Adding: 'Home'


/home/codespace/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


No chunks found
Created new chunk (9b595): Home & Significance

Adding: 'Product'
No chunks found
Created new chunk (37774): Products

Adding: 'Services'
No chunks found
Created new chunk (16eee): Consumer Services

Adding: 'Recruitment'
No chunks found
Created new chunk (a6658): Recruitment Process

Adding: 'Contact'
No chunks found
Created new chunk (e53a3): Contact Methods

Adding: 'English'
No chunks found
Created new chunk (57d7a): Language & Communication

Adding: 'Our specialized teams'
No chunks found
Created new chunk (8c560): Specialized Teams

Adding: 'We combine strategy, technology, data science and design to solve the most complex business challenges.'
No chunks found
Created new chunk (4f59b): Business Solutions

Adding: 'WHO ARE WE?'
No chunks found
Created new chunk (4b95e): Identity & Characteristics

Adding: 'Our specialized teams lead the product design and development process for high growth companies.'
No chunks found
Created new chunk (ce751): Product Development

In [29]:
import re
pages = []
links = soup.find_all("a")
for link in links:
    if re.match(r'^/.', link.get("href")):
        pages.append(asyncio.get_event_loop().run_until_complete(main(url[:-1] + link.get("href"))))
        # print(soup.text)
        